# Collect demonstrations

Interactive notebook for collecting demonstrations

In [1]:
%matplotlib auto

import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pathlib
import json
import random
import socket
from datetime import datetime
from demonstration_recorder import DemonstrationRecorder

# the gamepad is only available on some machines, and only on Linux
available_gamepad = True
try:
    from robotcontrol.gamepad_controller import GamepadController
except ModuleNotFoundError:
    print("Approxeng module not found, cannot use gamepad")
    available_gamepad = False
    
from robotcontrol.keyboard_controller import KeyboardController
from robotcontrol.program_controller import ProgramController
from robot.al5d_position_controller import PositionController, RobotPosition
from robot.al5d_simulated_position_controller import SimulatedPositionController


from camera.camera_controller import CameraController
from demonstration import Demonstration


Using matplotlib backend: module://matplotlib_inline.backend_inline
Approxeng module not found, cannot use gamepad


In [2]:
experiment = "demonstration_collector"
# picking up a collection specific name
hostname = socket.gethostname()
print(f"Hostname is {hostname}")
if hostname == "raven":
    run = "collect_freeform_raven"
elif hostname == "szenes.local":
    run = "collect_no_robot"
else:
    run = "collect_freeform"

exp = Config().get_experiment(experiment, run)
exp_target = Config().get_experiment("demonstration", exp["demonstration_name"])
exp_position_controller = Config().get_experiment(
    exp["exp_position_controller"], 
    exp["run_position_controller"])

exp_camera_controller = Config().get_experiment(exp["exp_camera_controller"], exp["run_camera_controller"])
exp_keyboard_controller = Config().get_experiment(exp["exp_keyboard_controller"], exp["run_keyboard_controller"])
exp_gamepad_controller = Config().get_experiment(exp["exp_gamepad_controller"], exp["run_gamepad_controller"])

Hostname is szenes.local
***ExpRun**: Loading pointer config file:
	/Users/lboloni/.config/BerryPicker/mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	/Users/lboloni/Google Drive/My Drive/LotziStudy/Code/PackageTracking/BerryPicker/settings/settings-szenes.yaml
***ExpRun**: Configuration for exp/run: demonstration_collector/collect_no_robot successfully loaded
***ExpRun**: Experiment default config /Users/lboloni/Documents/Develop/Github/BerryPicker/BerryPicker/src/experiment_configs/demonstration/_defaults_demonstration.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: demonstration/freeform successfully loaded
***ExpRun**: Configuration for exp/run: robot_al5d/position_controller_00 successfully loaded
***ExpRun**: Experiment default config /Users/lboloni/Documents/Develop/Github/BerryPicker/BerryPicker/src/experiment_configs/controllers/_defaults_controllers.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: controllers/camera_cam0_controller

In [3]:
# starting the robot controller
if exp["available_robot"]:
    robot_controller = PositionController(exp_position_controller)
else:
    robot_controller = SimulatedPositionController(exp_position_controller)
# starting the camere controller
camera_controller = CameraController(exp_camera_controller)
camera_controller.visualize = True


task_dir = exp_target.data_dir()
demoname = datetime.now().strftime('%Y_%m_%d__%H_%M_%S')
demo_dir = pathlib.Path(task_dir, demoname)
demo_dir.mkdir(parents=False, exist_ok=False)
print(f"Ok, demonstration with go into\n\t {demo_dir}")

demonstration = Demonstration(exp_target, demoname, parse_old_style = False)
demonstration.metadata["cameras"] = sorted(camera_controller.capture_devs.keys())
demonstration.save_metadata()
# fixme need to add the cameras into the demonstration

Ok, demonstration with go into
	 /Users/lboloni/Documents/Develop/Data/BerryPicker/demonstration/freeform/2025_08_11__12_59_48


OpenCV: out device of bound (0-1): 2
OpenCV: camera failed to properly initialize!


In [4]:
# Performs the recording. The robot is controlled through the primary 
# controller. This is also used to terminate the collection
# The information is collected by the demonstration recorder

controller = exp["primary_controller"]

if controller == "xbox":
    gamepad_controller = GamepadController(exp_gamepad_controller,
        robot_controller=robot_controller, camera_controller=camera_controller)
    demo_recorder = DemonstrationRecorder(demonstration,
        camera_controller=camera_controller, remote_control=gamepad_controller, robot_controller=
                                robot_controller, save_dir=demo_dir)
    # dr = None
    gamepad_controller.demonstration_recorder = demo_recorder
    gamepad_controller.control()
    demonstration.save_metadata()
    print("====== Demonstration terminated and recorded successfully, bye. ======")
if controller == "keyboard":
    kb_controller = KeyboardController(exp_keyboard_controller,
        robot_controller=robot_controller, camera_controller=camera_controller)
    demo_recorder = DemonstrationRecorder(demonstration, camera_controller=camera_controller, remote_control=kb_controller, robot_controller=
                                robot_controller, save_dir=demo_dir)
    # dr = None
    kb_controller.demonstration_recorder = demo_recorder
    kb_controller.control()
    demonstration.save_metadata()
    print("====== Demonstration terminated and recorded successfully, bye. ======")

if controller == "program":
    program_controller = ProgramController(
        robot_controller=robot_controller, camera_controller=camera_controller)
    # create wpcount 
    waypoints = []
    wpcount = 10
    while True:
        norm = [0] * 6
        for df in range(6):
            norm[df] = random.random()                
        rp = RobotPosition.from_normalized_vector(exp_position_controller, norm)
        if RobotPosition.limit(exp_position_controller, rp):
            print(f"added waypoint {rp}")
            waypoints.append(rp)
        if len(waypoints) >= wpcount:
            break

    program_controller.waypoints = waypoints
    program_controller.interactive_confirm = False
    demo_recorder = DemonstrationRecorder(demonstration, camera_controller=camera_controller, remote_control=program_controller, robot_controller=
                                robot_controller, save_dir=demo_dir)
    program_controller.demonstration_recorder = demo_recorder
    demonstration.save_metadata()
    program_controller.control()


INFO:robotcontrol.abstract_controller:***AbstractController***: Control robot: move to position Position:  height:4.90 distance:5.10 heading:0.21 wrist_angle:-45.10 wrist_rotation:75.50 gripper:95.00
INFO:robotcontrol.abstract_controller:***AbstractController***: Control robot done.
INFO:robotcontrol.abstract_controller:***AbstractController***: Update started
INFO:robotcontrol.abstract_controller:***AbstractController***: Update done


added waypoint Position:  height:4.07 distance:8.43 heading:0.21 wrist_angle:-45.10 wrist_rotation:87.75 gripper:9.84
added waypoint Position:  height:2.67 distance:7.55 heading:-56.84 wrist_angle:-40.46 wrist_rotation:78.92 gripper:36.39
added waypoint Position:  height:4.88 distance:5.95 heading:73.49 wrist_angle:-51.58 wrist_rotation:65.87 gripper:37.20
added waypoint Position:  height:4.44 distance:8.99 heading:64.35 wrist_angle:-72.92 wrist_rotation:72.52 gripper:59.92
added waypoint Position:  height:4.23 distance:3.36 heading:-21.69 wrist_angle:-61.93 wrist_rotation:87.15 gripper:91.72
added waypoint Position:  height:2.20 distance:7.98 heading:-75.30 wrist_angle:-55.11 wrist_rotation:74.63 gripper:50.79
added waypoint Position:  height:4.08 distance:3.84 heading:-18.31 wrist_angle:-53.18 wrist_rotation:73.26 gripper:85.20
added waypoint Position:  height:1.66 distance:6.97 heading:31.86 wrist_angle:-49.78 wrist_rotation:68.17 gripper:21.85
added waypoint Position:  height:2.09 

INFO:robotcontrol.abstract_controller:***AbstractController***: Control robot: move to position Position:  height:4.80 distance:5.20 heading:0.21 wrist_angle:-45.10 wrist_rotation:76.00 gripper:90.00
INFO:robotcontrol.abstract_controller:***AbstractController***: Control robot done.
INFO:robotcontrol.abstract_controller:***AbstractController***: Update started
INFO:robotcontrol.abstract_controller:***AbstractController***: Update done


Error at visualization? 
wp Position:  height:4.07 distance:8.43 heading:0.21 wrist_angle:-45.10 wrist_rotation:87.75 gripper:9.84
Distance to wp 0.32376207907994586
Error at visualization? 


INFO:robotcontrol.abstract_controller:***AbstractController***: Control robot: move to position Position:  height:4.70 distance:5.30 heading:0.21 wrist_angle:-45.10 wrist_rotation:76.50 gripper:85.00
INFO:robotcontrol.abstract_controller:***AbstractController***: Control robot done.
INFO:robotcontrol.abstract_controller:***AbstractController***: Update started
INFO:robotcontrol.abstract_controller:***AbstractController***: Update done


...

Error at visualization? 
wp Position:  height:3.02 distance:3.06 heading:-58.54 wrist_angle:-30.40 wrist_rotation:71.84 gripper:2.20
Distance to wp 0.006511493275562922
Error at visualization? 


INFO:robotcontrol.abstract_controller:***AbstractController***: Control robot: move to position Position:  height:3.02 distance:3.13 heading:-58.54 wrist_angle:-30.40 wrist_rotation:71.84 gripper:2.20
INFO:robotcontrol.abstract_controller:***AbstractController***: Control robot done.
INFO:robotcontrol.abstract_controller:***AbstractController***: Update started
INFO:robotcontrol.abstract_controller:***AbstractController***: Update done


Error at visualization? 
wp Position:  height:3.02 distance:3.06 heading:-58.54 wrist_angle:-30.40 wrist_rotation:71.84 gripper:2.20
Distance to wp 0.0041305410365263615
Error at visualization? 


INFO:robotcontrol.abstract_controller:***AbstractController***: Control robot: move to position Position:  height:3.02 distance:3.06 heading:-58.54 wrist_angle:-30.40 wrist_rotation:71.84 gripper:2.20
INFO:robotcontrol.abstract_controller:***AbstractController***: Control robot done.
INFO:robotcontrol.abstract_controller:***AbstractController***: Update started
INFO:robotcontrol.abstract_controller:***AbstractController***: Update done


Error at visualization? 
wp Position:  height:3.02 distance:3.06 heading:-58.54 wrist_angle:-30.40 wrist_rotation:71.84 gripper:2.20
Distance to wp 0.001749588952710231
Error at visualization? 
Error at visualization? 
wp Position:  height:3.02 distance:3.06 heading:-58.54 wrist_angle:-30.40 wrist_rotation:71.84 gripper:2.20
Distance to wp 0.0
Finished waypoints
stop_robot on the simulated robot


In [5]:
demonstration.save_metadata()
demonstration.move_to_video(delete_img_files=True)


***Demonstration***: moving to video started
***Demonstration***: moving to video done


In [6]:
print(demonstration.metadata)

{'stored_as_video': True, 'stored_as_images': False, 'maxsteps': 464, 'cameras': []}


In [7]:
exp

Experiment:
    available_robot: false
    data_dir: /Users/lboloni/Documents/Develop/Data/BerryPicker/demonstration_collector/collect_no_robot
    demonstration_name: freeform
    exp_camera_controller: controllers
    exp_gamepad_controller: controllers
    exp_keyboard_controller: controllers
    exp_position_controller: robot_al5d
    exp_run_sys_indep_file: /Users/lboloni/Documents/Develop/Github/BerryPicker/BerryPicker/src/experiment_configs/demonstration_collector/collect_no_robot.yaml
    experiment_name: demonstration_collector
    primary_controller: program
    run_camera_controller: camera_cam0_controller
    run_gamepad_controller: gamepad_x360_controller
    run_keyboard_controller: keyboard_controller
    run_name: collect_no_robot
    run_position_controller: position_controller_00
    subrun_name: null